In [2]:
pip install nltk

In [4]:
pip install torch torchvision


In [17]:
pip install nltk pyarabic


In [25]:
import requests
from bs4 import BeautifulSoup

# Fonction pour extraire du texte en arabe depuis un site web
def scrape_arabic_text(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, "html.parser")
        paragraphs = soup.find_all('p')  # Récupérer les balises <p>
        arabic_texts = [p.text.strip() for p in paragraphs if p.text.strip()]
        return arabic_texts
    except Exception as e:
        print(f"Erreur lors du scraping de {url}: {e}")
        return []

# Liste de sites web en arabe sur un sujet donné
websites = [
    "https://arabic.cnn.com/",
    "https://www.aljazeera.net/",
    "https://www.skynewsarabia.com/"
]

# Rassembler tous les textes extraits
all_texts = []
for site in websites:
    print(f"Scraping du site : {site}")
    texts = scrape_arabic_text(site)
    all_texts.extend(texts)

# Limiter aux 10 premiers textes
limited_texts = all_texts[:10]


Scraping du site : https://arabic.cnn.com/
Scraping du site : https://www.aljazeera.net/
Scraping du site : https://www.skynewsarabia.com/


In [26]:
import nltk
from nltk.corpus import stopwords
from pyarabic.araby import tokenize, strip_tashkeel, normalize_alef
from nltk.stem.isri import ISRIStemmer

# Télécharger les ressources nécessaires pour NLTK
nltk.download('stopwords')

# Étape 1 : Préparer les stop words en arabe
arabic_stopwords = set(stopwords.words("arabic"))

# Ajouter des stop words spécifiques si nécessaire
custom_stopwords = {"و", "في", "من", "على", "أن", "ما"}  # Exemples
arabic_stopwords.update(custom_stopwords)

# Étape 2 : Définir un stemmer arabe (ISRI Stemmer)
stemmer = ISRIStemmer()

# Fonction de prétraitement NLP
def preprocess_text(text):
    # a. Normalisation du texte
    text = strip_tashkeel(text)  # Retirer les diacritiques (tashkeel)
    text = normalize_alef(text)  # Normaliser les différentes formes d'alef (أ → ا)

    # b. Tokenisation avec pyarabic
    tokens = tokenize(text)

    # c. Suppression des stop words
    tokens = [word for word in tokens if word not in arabic_stopwords]

    # d. Stemming
    stemmed_tokens = [stemmer.stem(word) for word in tokens]

    # e. Reconstituer le texte après prétraitement
    processed_text = " ".join(stemmed_tokens)
    return processed_text

# Appliquer le prétraitement aux textes
processed_texts = [preprocess_text(text) for text in limited_texts]
print(processed_texts)


['اعل عرض سور يطر علا قرا بلد عدة ريف حمة شرق غرب ، وبن اكد انه قرب دين حمة ركز حفظ ، قلت وزر دفع سور ان قوت وصل ضرب “ نظم رهب ”.', 'يرا حلل ان تفق حرك قام سلم ( حمس ) حرر وطن لسط ( فتح ) علا شكل لجن شرك ادر قطع غزة سيك دخل بتج وقف حرب اسرائيلية علا قطع .', 'افد حقق هارتس ، بان وجة عدا سمي عقب طوف اقص ، دفع غرب اسرائيلي بدي سيل الا خرج عزل ، وتب نشط شعب حلي لرد علا سمه “ شيط ” رائيل يهد .', 'تهم دير شفا كمل عدو شمل غزة جيش اسرائيلي باستخدام شفا كقعد عسكر ، حدث هول شفا عرض طبء عمل رضا .', 'قلت وند ان حكم سور دفع ثمن رفض قدم نزل عدم قدر علا صلح خرج عزل ،“ فعد مرد الا نعش ان يبد جيش حلفاؤ يرن حزب الل اي قام ”.', 'قلت برس رنس ان قوت جيش روس قدم كثر 725 كيلومترا ربع دخل رضي اوكرانية خلل شهر / ثني اضي مثل تقد سبق كثر عام شرق وكر .', 'ورد يمز بريطانية ان عدد هجر نظم يقم رحل خطر عبر قنة انجليزية بلغ كثر 20 الف تول حزب عمل سلط ، بعد عهد لند شدد قعد هجر .', 'يكن علماء سلف علا ذهب فقه عمل سلط جئر ؛ جمع اهل حدث ارخ عرق عرض سيس تصل احا الا حد ثور . وفي قال عرض اهم جرب ارخ .', 'صوت رلم اسرائيلي ، يو

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [27]:
# Vocabulaire d'exemple
vocab = {"<PAD>": 0, "هذا": 1, "مثال": 2, "نص": 3, "يحتوي": 4, "على": 5, "معلومات": 6, "حول": 7, "الموضوع": 8, "إضافية": 9, "بيانات": 10, "تفاصيل": 11}
max_len = 10  # Limite de longueur pour les séquences

# Tokenisation et padding
def tokenize_and_pad(text, vocab, max_len):
    tokens = [vocab.get(word, vocab["<PAD>"]) for word in text.split()]
    tokens = tokens[:max_len]  # Truncate if too long
    tokens += [vocab["<PAD>"]] * (max_len - len(tokens))  # Pad if too short
    return torch.tensor(tokens)

# Créer un Dataset personnalisé
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, texts, scores, vocab, max_len):
        self.texts = [tokenize_and_pad(text, vocab, max_len) for text in texts]
        self.scores = torch.tensor(scores, dtype=torch.float32)

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx], self.scores[idx]

# Dataset et DataLoader
scores = [6.5, 7.0, 8.0, 7.5, 8.5, 6.0, 7.8, 8.2, 9.0, 7.1]  # Exemple de scores
dataset = TextDataset(processed_texts, scores, vocab, max_len)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=2, shuffle=True)


In [28]:
class RNNModel(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(RNNModel, self).__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embedding_dim)
        self.rnn = torch.nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        embedded = self.embedding(x)
        output, _ = self.rnn(embedded)
        output = self.fc(output[:, -1, :])  # Use the last output
        return output


In [29]:
class BiRNNModel(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(BiRNNModel, self).__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embedding_dim)
        self.rnn = torch.nn.RNN(embedding_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.fc = torch.nn.Linear(2 * hidden_dim, output_dim)  # *2 for bidirectional

    def forward(self, x):
        embedded = self.embedding(x)
        output, _ = self.rnn(embedded)
        output = self.fc(output[:, -1, :])  # Use the last output
        return output


In [30]:
class GRUModel(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(GRUModel, self).__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embedding_dim)
        self.gru = torch.nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        embedded = self.embedding(x)
        output, _ = self.gru(embedded)
        output = self.fc(output[:, -1, :])  # Use the last output
        return output


In [31]:
class LSTMModel(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embedding_dim)
        self.lstm = torch.nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        embedded = self.embedding(x)
        output, _ = self.lstm(embedded)
        output = self.fc(output[:, -1, :])  # Use the last output
        return output


In [37]:
# Initialisation du modèle, optimiseur et critère
model = LSTMModel(len(vocab), embedding_dim=50, hidden_dim=64, output_dim=1)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.MSELoss()

# Boucle d'entraînement
epochs = 15
for epoch in range(epochs):
    model.train()  # Mettre le modèle en mode entraînement
    epoch_loss = 0
    for texts, scores in dataloader:
        optimizer.zero_grad()  # Réinitialiser les gradients à zéro

        # Passage des données à travers le modèle
        outputs = model(texts)

        # Calcul de la perte
        loss = criterion(outputs.squeeze(), scores)

        # Rétropropagation de la perte
        loss.backward()

        # Mise à jour des poids
        optimizer.step()

        # Accumulation de la perte pour chaque lot
        epoch_loss += loss.item()

    # Affichage de la perte moyenne pour l'époque
    print(f'Époque {epoch + 1}/{epochs}, Perte : {epoch_loss / len(dataloader):.4f}')


Époque 1/15, Perte : 52.8088
Époque 2/15, Perte : 41.9922
Époque 3/15, Perte : 31.7049
Époque 4/15, Perte : 23.7934
Époque 5/15, Perte : 17.4663
Époque 6/15, Perte : 12.4519
Époque 7/15, Perte : 8.7476
Époque 8/15, Perte : 5.8837
Époque 9/15, Perte : 3.9780
Époque 10/15, Perte : 2.7782
Époque 11/15, Perte : 1.9748
Époque 12/15, Perte : 1.4151
Époque 13/15, Perte : 1.1032
Époque 14/15, Perte : 0.9268
Époque 15/15, Perte : 0.8612


In [38]:
# Tester le modèle sur de nouvelles données (exemple)
model.eval()  # Passer le modèle en mode évaluation
new_texts = ["نص جديد للاختبار"]
new_processed_texts = [preprocess_text(text) for text in new_texts]
new_texts_tensor = torch.stack([tokenize_and_pad(text, vocab, max_len) for text in new_processed_texts])

# Prédiction
with torch.no_grad():  # Pas besoin de gradients lors de l'inférence
    predictions = model(new_texts_tensor)
    print("Prédictions :", predictions)


Prédictions : tensor([[7.3399]])


In [39]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# Supposons que `predictions` contienne les prédictions de votre modèle et `scores` les scores réels.
# Exemple de prédictions et scores réels (ces valeurs doivent provenir de vos données de test).
real_scores = np.array([6.5, 7.0, 8.0, 7.5, 8.5, 6.0, 7.8, 8.2, 9.0, 7.1])  # Scores réels
predictions = np.array([6.2, 6.8, 8.1, 7.3, 8.2, 5.9, 7.9, 8.0, 8.8, 7.2])  # Prédictions du modèle

# Calcul de l'Erreur Quadratique Moyenne (MSE)
mse = mean_squared_error(real_scores, predictions)
print(f"Erreur Quadratique Moyenne (MSE) : {mse:.4f}")

# Calcul de l'Erreur Absolue Moyenne (MAE)
mae = mean_absolute_error(real_scores, predictions)
print(f"Erreur Absolue Moyenne (MAE) : {mae:.4f}")

# Calcul de la racine de l'Erreur Quadratique Moyenne (RMSE)
rmse = np.sqrt(mse)
print(f"Racine de l'Erreur Quadratique Moyenne (RMSE) : {rmse:.4f}")


Erreur Quadratique Moyenne (MSE) : 0.0380
Erreur Absolue Moyenne (MAE) : 0.1800
Racine de l'Erreur Quadratique Moyenne (RMSE) : 0.1949


In [40]:
from nltk.translate.bleu_score import corpus_bleu

# Exemple de texte généré et de texte de référence
# Le texte généré par le modèle
generated_texts = [
    "هذا مثال عن نص باللغة العربية",
    "نص آخر يحتوي على معلومات مفيدة",
    "مثال ثالث لتحليل البيانات"
]

# Le texte de référence (pour évaluer la qualité)
reference_texts = [
    ["هذا مثال على نص باللغة العربية"],
    ["نص آخر يحتوي على معلومات قيمة"],
    ["مثال ثالث لتحليل معلومات البيانات"]
]

# Calcul du score BLEU
bleu_score = corpus_bleu(reference_texts, generated_texts)
print(f"BLEU score : {bleu_score:.4f}")


BLEU score : 0.8210


In [42]:
# Prédictions des modèles (exemples de texte généré par les modèles)
model_predictions_rnn = [
    "هذا مثال عن نص باللغة العربية",
    "نص آخر يحتوي على معلومات مفيدة",
    "مثال ثالث لتحليل البيانات"
]

model_predictions_gru = [
    "هذا نص يحتوي على معلومات قيمة",
    "نص آخر مختصر حول الموضوع",
    "معلومات إضافية لتحليل البيانات"
]

model_predictions_lstm = [
    "نص مع بعض التفاصيل المهمة",
    "مثال آخر يحتوي على كلمات مفيدة",
    "نص من أجل تقديم تفاصيل"
]

# Références (exemples de texte de référence pour évaluer les prédictions)
reference_texts = [
    ["هذا مثال على نص باللغة العربية"],
    ["نص آخر يحتوي على معلومات قيمة"],
    ["مثال ثالث لتحليل البيانات"]
]

# Calcul du BLEU score pour chaque modèle
from nltk.translate.bleu_score import corpus_bleu

bleu_rnn = corpus_bleu(reference_texts, model_predictions_rnn)
bleu_gru = corpus_bleu(reference_texts, model_predictions_gru)
bleu_lstm = corpus_bleu(reference_texts, model_predictions_lstm)

# Affichage des résultats
print("Évaluation des modèles :")
print(f"RNN - BLEU: {bleu_rnn:.4f}")
print(f"GRU - BLEU: {bleu_gru:.4f}")
print(f"LSTM - BLEU: {bleu_lstm:.4f}")


Évaluation des modèles :
RNN - BLEU: 0.9133
GRU - BLEU: 0.4045
LSTM - BLEU: 0.3045


In [44]:
!pip install transformers

In [45]:
!pip install torch

In [46]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Charger le modèle GPT-2 et son tokenizer
model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Assurez-vous que le modèle est bien en mode évaluation
model.eval()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [48]:
from transformers import GPT2Tokenizer

# Charger le tokenizer de GPT-2
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Ajouter un token de padding (utiliser le même token que EOS)
tokenizer.pad_token = tokenizer.eos_token  # Utilise eos_token comme pad_token


In [49]:
# Exemple de données d'entraînement
train_texts = [
    "L'intelligence artificielle est un domaine fascinant.",
    "Les réseaux de neurones sont à la base de nombreuses avancées en IA.",
    "Le deep learning est une branche du machine learning.",
    "GPT-2 est un modèle de langage pré-entraîné utilisé pour la génération de texte.",
    "Transformer est un modèle d'architecture populaire dans les réseaux neuronaux."
]

# Tokenisation des données d'entraînement
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=50)

# Convertir en Dataset
import torch
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings['input_ids'])

train_dataset = TextDataset(train_encodings)


In [52]:
from transformers import Trainer, TrainingArguments

# Définir les arguments d'entraînement
training_args = TrainingArguments(
    output_dir='./results',          # Répertoire de sortie
    num_train_epochs=3,              # Nombre d'époques
    per_device_train_batch_size=2,   # Taille du batch
    logging_dir='./logs',            # Répertoire pour les logs
)

# Utiliser Trainer pour fine-tuner GPT-2
trainer = Trainer(
    model=model,                         # Le modèle pré-entraîné
    args=training_args,                  # Arguments d'entraînement
    train_dataset=train_dataset,         # Dataset d'entraînement
)

# Fine-tuning
trainer.train()


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


ValueError: The model did not return a loss from the inputs, only the following keys: logits,past_key_values. For reference, the inputs it received are input_ids,attention_mask.